In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/dtu/blackhole/02/186978/cache'


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_math = AutoModelForCausalLM.from_pretrained("meta-math/MetaMath-Mistral-7B")
model_bio = AutoModelForCausalLM.from_pretrained("BioMistral/BioMistral-7B")
tokenizer_bio = AutoTokenizer.from_pretrained("BioMistral/BioMistral-7B")
tokenizer_math = AutoTokenizer.from_pretrained("meta-math/MetaMath-Mistral-7B")

/zhome/30/d/186978/DL/lib64/python3.9/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
input_text = "Solve the equation x^2 - 4x + 4 = 0"
inputs = tokenizer_math(input_text, return_tensors="pt")
outputs = model_math.generate(**inputs)
print(tokenizer_math.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/zhome/30/d/186978/DL/lib64/python3.9/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Solve the equation x^2 - 4x + 4 = 0.



In [4]:
import torch

base_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
base_model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
if len(tokenizer_math) < len(base_tokenizer):
    tokenizer_math.add_tokens(["<dummy>"])
    model_math.resize_token_embeddings(len(tokenizer_math))
elif len(base_tokenizer) < len(tokenizer_math):
    base_tokenizer.add_tokens(["<dummy>"])
    base_model.resize_token_embeddings(len(base_tokenizer))
    

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [6]:
if len(tokenizer_bio) < len(base_tokenizer):
    tokenizer_bio.add_tokens(["<dummy>"])
    model_bio.resize_token_embeddings(len(tokenizer_bio))

In [7]:
len(tokenizer_math), len(tokenizer_bio), len(base_tokenizer)

(32001, 32001, 32001)

In [ ]:
# def extract_deltas(model, base_model):
#     deltas = {}
#     for name, param in model.named_parameters():
#         base_param = base_model.state_dict()[name]
#         deltas[name] = param - base_param
#     return deltas

In [ ]:
import torch

def extract_deltas(model, base_model):
    deltas = {}
    base_state_dict = base_model.state_dict()
    
    for name, param in model.named_parameters():
        if name in base_state_dict:
            base_param = base_state_dict[name]
            if param.shape == base_param.shape:
                # Detach tensors to avoid gradient issues
                deltas[name] = (param.detach() - base_param.detach()).cpu()
            else:
                print(f"Skipping {name} due to shape mismatch: {param.shape} vs {base_param.shape}")
        else:
            print(f"Skipping {name} as it is not in the base model")
    
    return deltas

: 

In [9]:
deltas_math = extract_deltas(model_math, base_model)
deltas_bio = extract_deltas(model_bio, base_model)

: 

: 

In [18]:
import numpy as np

def apply_dare(deltas, drop_rate):
    new_deltas = {}
    for name, delta in deltas.items():
        # Detach the tensor from the computation graph and convert to NumPy array
        mask = np.random.rand(*delta.shape) > drop_rate
        mask = torch.tensor(mask, dtype=delta.dtype, device=delta.device)
        new_deltas[name] = delta * mask / (1 - drop_rate)
    return new_deltas

In [19]:
drop_rate = 0.9
dare_deltas_math = apply_dare(deltas_math, drop_rate)
dare_deltas_bio = apply_dare(deltas_bio, drop_rate)

: 

: 

: 